# Zarr (v2) store reading with xtensor-zarr

In this notebook we use [zarr-python](https://github.com/zarr-developers/zarr-python) to chunk an array and store it in a store (in Zarr v2 format). We then read it with [xtensor-zarr](https://github.com/xtensor-stack/xtensor-zarr), which supports reading Zarr v2 stores.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.image import imread
import zarr

In [ ]:
a = imread('marie_curie.jpg')

In [ ]:
a

In [ ]:
plt.imshow(a, cmap='Greys_r')

In [ ]:
a.shape

## Store the array in Zarr v2

In [ ]:
!rm -rf marie_curie.zarr

In [ ]:
z = zarr.open('marie_curie.zarr', mode='w', shape=a.shape, chunks=(30, 20), dtype=a.dtype)

In [ ]:
z[:] = a

In [ ]:
!ls marie_curie.zarr

The array consists of about 100 chunks.

In [ ]:
>xcpp14
#include "xtensor/xio.hpp"
#include "xtensor/xview.hpp"
#include "xwidgets/ximage.hpp"
#include "xtensor/xchunked_array.hpp"
#include "xtensor-io/xio_blosc.hpp"
#include "xtensor-io/xchunk_store_manager.hpp"
#include "xtensor-io/ximage.hpp"
#include "xtensor-zarr/xzarr_hierarchy.hpp"
#include "xtensor-zarr/xzarr_file_system_store.hpp"
#include "xtensor-zarr/xtensor_zarr_config_cling.hpp"
#include "xtl/xbase64.hpp"
#include "xcpp/xdisplay.hpp"

## Read the Zarr store (v2) with xtensor-zarr

In [ ]:
>xcpp14
xt::xzarr_register_compressor<xt::xzarr_file_system_store, xt::xio_blosc_config>();
auto h = xt::get_zarr_hierarchy("marie_curie.zarr");
auto z = h.get_array("");

In [ ]:
>xcpp14
auto a = z.get_array<uint8_t>();
a

In [ ]:
>xcpp14
std::vector<char> read_file(const char* filename)
{
    std::basic_ifstream<char> file(filename, std::ios::binary);
    return std::vector<char>((std::istreambuf_iterator<char>(file)), std::istreambuf_iterator<char>());
}

In [ ]:
>xcpp14
template <class E>
std::vector<char> to_png_buffer(const xt::xexpression<E>& e)
{
    const char* temp_filename = "/tmp/xio_image.png";
    xt::dump_image(temp_filename, e);
    return read_file(temp_filename);
}

In [ ]:
>xcpp14
xw::image img;
img.value = to_png_buffer(a);
img

The original image has been reconstructed.